In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as mpl

def mid(npArr):
    return npArr.sum() / len(npArr)

#Расчет косвенной погрешности, если величина - произведение и частное данных
def sigma_mulls(val, sigsArr, parmsArr):
    sum = 0
    for i in range(len(sigsArr)):
        sum += (sigsArr[i] / parmsArr[i])**2
    return val * (sum)**0.5


#Расчет случайной погрешности среднего
def sigma_rand(valsArr):
    N = len(valsArr)
    valMid = sum(valsArr) / N
    msum = 0
    for val in valsArr:
        msum += (val - valMid)**2
        
    return (msum / (N * (N - 1))) ** 0.5


#MNK если y = kx
##DEPRECATED
def k_only(x, y):
    return mid(x * y) / mid(x * x)

##DEPRECATED
def k_only_sigma(x, y):
    return ((mid(x * x) * mid(y * y) - mid(x * y)**2) / (len(x) * mid(x * x)**2))**0.5

#MNK для y = kx + a
def k_coef(x, y):
    return (mid(x * y) - mid(x) * mid(y)) / (mid(x * x) - mid(x)**2)

def b_coef(x, y):
    return mid(y) - k_coef(x, y) * mid(x)

def k_sigma(x, y):
    return ((mid(y * y) - mid(y)**2) / (mid(x * x) - mid(x)**2) - k_coef(x, y)**2)**0.5  / len(x)**0.5






In [ ]:
Fe = np.array ([139835., 139093., 139020.],
                [83247., 82020., 82922.],
                [46805., 46927., 46525.],
                [27098., 27109., 27499.],
                [15679., 15557., 15744.])
tmp = np.zeros (5)
FeSig = np.zeros (5)
for i in range (5):
    tmp[i] = mid (Fe[i])
    FeSig[i] = sigma_rand (Fe[i])
Fe = tmp


Al = np.array ([163533., 162998., 161773.],
                [110952., 111181., 110676.],
                [74503., 74995., 74282.],
                [48968., 49111., 49625.],
                [33057., 32998., 33555.])
tmp = np.zeros (5)
AlSig = np.zeros (5)
for i in range (5):
    tmp[i] = mid (Al[i])
    AlSig[i] = sigma_rand (Al[i])

Al = tmp

Pb = np.array ([137743., 136171., 137868.],
                [82532., 83044., 82591.],
                [50134., 50740., 50559.],
                [33414., 33418., 33191.],
                [22010., 21721., 22012.])
tmp = np.zeros (5)
PbSig = np.zeros (5)
for i in range (5):
    tmp[i] = mid (Pb[i])
    PbSig[i] = sigma_rand (Pb[i])
Pb = tmp

N0 = (261879 + 252786 + 248313) / 3



In [ ]:
#пересчёт в логарифмический масштаб
FeLogSig = FeSig / Fe
FeLog = np.log (Fe)

PbLogSig = PbSig / Pb
PbLog = np.log (Pb)

AlLogSig = AlSig / Al
AlLog = np.log (Al)

In [ ]:
mpl.rcParams['font.size'] = 20
    

fig, ax = plt.subplots()

ax.set_title("Логарифм числа частиц от толщины пройденного материала")
ax.set_ylabel(r"$ln(N)$")
ax.set_xlabel(r"$Толщина, ....$")

fig.set_figwidth(14)
fig.set_figheight(14)

ax.scatter(dists, throughputs)
ax.scatter(distsOld, throughputsOld)

k = k_coef(Carr, Tarr)
b = b_coef(Carr, Tarr)
xTwoPoint = np.linspace(0, Carr[-1], 2)
yForTwoPoint = k * xTwoPoint + b 


mStyle = ['--', '-', '-.', ':'] 

#ax.plot(xTwoPoint, yForTwoPoint, label=r'T = %f * C + %f'%(k, b), linestyle=mStyle[3])
ax.plot(dists, throughputs, label="now")
ax.plot(distsOld, throughputsOld, label="old")
# ax.errorbar(Carr, Tarr, yerr=0.02, xerr=0.0005, fmt='.') 

ax.minorticks_on()

ax.grid(which='major',
        color = 'k', 
        linewidth = 0.5,
       alpha = 0.5)

ax.grid(which='minor', 
        color = 'k', 
        linestyle = ':')

plt.legend()
plt.savefig('graph.jpeg')
plt.show()
